In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.bem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG

**keys**: homogeneous Neumann bvp, hypersingular operator, unknown Dirichlet data

Neumann Laplace Direct Method 
======================================================

Consider the Neumann boundary value problem 

$$ \left\{ \begin{array}{rlc l} \Delta u &=& 0, \quad &\Omega \subset \mathbb R^3\,,\\ \gamma_1 u&=& j, \quad &\Gamma = \partial \Omega\,.\end{array} \right. $$ 

The unique solution $u\in H^1(\Omega)$ can be written in the following form (representation forumula) 

$$ u(x) = \underbrace{ \int\limits_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{1}{\| x-y\|} } \, j(y)\, \mathrm{d}s_y}_{\displaystyle{ \mathrm{SL}(j) }} + \underbrace{ \int\limits_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{\langle n(y), x-y \rangle}{\| x-y\|^3} } \, m( y)\, \mathrm{d}s_y}_{\displaystyle{ \mathrm{DL}(m) }}\,.$$ 


**NGSolve Solution**

Define the geometry $\Omega \subset \mathbb R^3$ and create a mesh:

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.2)).Curve(2)

Define conforming finite element spaces for $H^{\frac12}(\Gamma)$ and $H^{-\frac12}(\Gamma)$ respectively:  

In [ ]:
fesL2 = SurfaceL2(mesh, order=2, dual_mapping=False) # TODO: check interpolation with option dual_mapping=True  
u,v = fesL2.TnT()
fesH1 = H1(mesh, order=2, definedon=mesh.Boundaries(".*"))
uH1,vH1 = fesH1.TnT()
print ("ndofL2 = ", fesL2.ndof, "ndof H1 = ", fesH1.ndof)

Project the given Neumann data $u_1$ in finite element space and have a look at the boundary data:

In [ ]:
uexa = 1/ sqrt( (x-1)**2 + (y-1)**2 + (z-1)**2 )

graduexa = CF( (uexa.Diff(x), uexa.Diff(y), uexa.Diff(z)) )
n = specialcf.normal(3)
j = GridFunction(fesL2)
j.Interpolate(graduexa*n, definedon=mesh.Boundaries(".*"))

Draw (j, mesh, draw_vol=False, order=3);

**Boundary Integral Equation**

Let's carefully apply the Neumann trace $\gamma_1$ to the representation formula and solve the resulting boundary integral equation for the Dirichlet trace $u_0$ of $u$ by discretisation of the following variational formulation: 

$$\begin{array}{r c ccc} \forall \, v\in H^{\frac12}(\Gamma): && \displaystyle \int\limits_\Gamma v \cdot \gamma_1 \left(\mathrm{DL}(m) \right) \, \mathrm d s  &=&  \displaystyle \int\limits_\Gamma j \cdot  v \,\mathrm d s &-& \displaystyle \int\limits_\Gamma v \cdot \gamma_1 \left(\mathrm{SL}(j)\right) \, \mathrm d s \\[2ex] & \Rightarrow & \mathrm{D}\,\mathrm{m} &=& \dfrac12 \, \mathrm{M} \, \mathrm j &-& \mathrm{K}' \, \mathrm j\,. \end{array} $$ 


The discretisation of the variational formulation leads to a system of linear equations, ie 

$$ \left( \mathrm{D} + \mathrm{S}\right) \, \mathrm{m} = \left( \frac12 \,\mathrm{M} - \mathrm{K}' \right) \, \mathrm{j} \,, $$

where the linear operators are as follows
- $\mathrm{S}$ is a stabilisation matrix that removes the one dimensional kernel from hypersinbular operator.
-  $\mathrm{D}$ is the hypersingular operator and the stabilisation $(\mathrm D + \mathrm{S})$ is regular and symmetric.
- $\mathrm{M}$ is a mass matrix.
- $\mathrm{K}'$ is the adjoint double layer operator. 

The stabilisation matrix $S$ is needed to cope with the kernel of the hypersingular operator. The following stabilisation matrix is used here: 

$$ S \in \mathbb R^{n\times n}, \quad S_{ij} = \int\limits_{\Gamma} v_i(x) \,\mathrm{d} s \cdot \int\limits_{\Gamma} v_j(x) \,\mathrm{d} s\,,$$ 

where $v_i, v_j$ being $H^{\frac12}(\Gamma)$-conforming shape functions. 

In [ ]:
vH1m1 = LinearForm(vH1*1*ds(bonus_intorder=3)).Assemble()
S = (BaseMatrix(Matrix(vH1m1.vec.Reshape(1))))@(BaseMatrix(Matrix(vH1m1.vec.Reshape(fesH1.ndof))))

The assembly of the hypersingular operator is declarative and close to their actual implementations (explicit formula are given below).

In [ ]:
with TaskManager(): 
    D = LaplaceSL(curl(uH1)*ds)*curl(vH1)*ds
    K = LaplaceDL(uH1*ds)*v*ds
    M = BilinearForm( uH1 * v * ds(bonus_intorder=3)).Assemble()

Left and right hand side of linear system are computed and the solution is the Dirichlet data $u_0$:

In [ ]:
m = GridFunction(fesH1)
pre = BilinearForm(uH1*vH1*ds).Assemble().mat.Inverse(freedofs=fesH1.FreeDofs())
with TaskManager(): 
    lhs = D.mat + S
    rhs = ( (0.5 * M.mat.T - K.mat.T) * j.vec ).Evaluate()
    CG(mat=lhs, pre=pre, rhs=rhs, sol=m.vec, tol=1e-8, maxsteps=200, initialize=False, printrates=True)

Let's have a look at the numerical and exact Dirichlet data and compare them quantitatively:

In [ ]:
Draw (m, mesh, draw_vol=False, order=3);
mexa = GridFunction(fesH1)
mexa.Interpolate (uexa, definedon=mesh.Boundaries(".*"))
Draw (mexa, mesh, draw_vol=False, order=3);

In [ ]:
print ("L2 error of surface gradients =", sqrt (Integrate ( (grad(mexa)-grad(m))**2, mesh, BND)))

**TODO Evaluation Solution**

**Details**
  
The hypersingular operator is implemented as follows (reference below)

$$ D_{ij} = \langle v_j, \gamma_1 \mathrm{DL}(u_i) \rangle_{-\frac12} = \frac{1}{4\pi} \int\limits_\Gamma\int\limits_\Gamma \frac{ \langle \mathrm{\boldsymbol {curl}}_\Gamma  \,u_i(x), \mathrm{\boldsymbol{curl}}_\Gamma\, v_j(y) \rangle}{\|x-y\|} \, \mathrm{d} s_{y} \, \mathrm{d} s_x $$ 

This explicit represenation of $D$ is the singular layer potential operator with vector valued surface curl operator applied on trial and test functions. You may assemble $D$ it by

```
D = LaplaceSL(curl(u)*ds)*curl(v)*ds
```

**References:** 

For details on the explicit representation of the hypersingular operator have a look into [Numerische Näherungsverfahren für elliptische Randwertprobleme](https://link.springer.com/book/10.1007/978-3-322-80054-1), p.127, p.259 (1st edition).